In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
import numpy as np

from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

from google.colab import files


In [0]:
data_path = os.path.join("/content/gdrive/My Drive/", "DRU-MAWI-project/ICHI14_dataset/data")
patient_list = ['002','003','005','007','08a','08b','09a','09b', '10a','011','013','014','15a','15b','016',
            '017','018','019','020','021','022','023','025','026','027','028','029','030','031','032',
            '033','034','035','036','037','038','040','042','043','044','045','047','048','049','051']

In [0]:
train_patient_list, test_patient_list = train_test_split(patient_list, random_state=100, test_size=0.3)
test_patient_list, valid_patient_list = train_test_split(test_patient_list, random_state=100, test_size=0.5)

In [5]:
print(len(patient_list))
print(len(train_patient_list))
print(len(valid_patient_list))
print(len(test_patient_list))

45
31
7
7


In [6]:
print(train_patient_list)
print(valid_patient_list)
print(test_patient_list)

['022', '09b', '048', '020', '023', '15b', '003', '042', '15a', '038', '025', '011', '018', '029', '031', '014', '08a', '047', '049', '016', '040', '005', '037', '033', '013', '017', '026', '044', '007', '027', '10a']
['035', '034', '051', '019', '045', '043', '08b']
['021', '09a', '002', '028', '032', '036', '030']


In [0]:
def change_labels(sample):
    """
    Returns:
    sample - contains only label 1(awake) and 0(sleep) for polisomnography
    """
    
    sample.gt[sample.gt==0] = 8
    sample.gt[np.logical_or.reduce((sample.gt==1, sample.gt==2, sample.gt==3, sample.gt==5))] = 0
    sample.gt[np.logical_or.reduce((sample.gt==6, sample.gt==7, sample.gt==8))] = 1
    
    return sample   

#-------------------------------------------------------------------------

def decoder(sample):
    '''
    Returns: 
    decoded_sample - contains accelerometer and ps data for each sensor record, ndarray of shape (n_records, 4)
    
    '''

    sample = np.repeat(sample, sample.d, axis=0)
    n_records = sample.shape[0]
    decoded_sample = np.zeros((n_records, 4))
    
    decoded_sample[:, 0] = sample.x
    decoded_sample[:, 1] = sample.y
    decoded_sample[:, 2] = sample.z
    decoded_sample[:, 3] = sample.gt
    
    return decoded_sample

#-------------------------------------------------------------------------

def divide_by_windows(decoded_sample, window_len=60):
    """
    Parameters:
    wondow_len - length of each window in seconds, int
    Returns:
    X - accelerometer data, ndarray of shape (n_windows, window_len, 3)
    y - polisomnography data, ndarray of shape (n_windows, )
    """
    
    window_len *= 100
    n_windows = decoded_sample.shape[0] // window_len
    
    X = np.zeros((n_windows, window_len, 3))
    y = np.zeros(n_windows)
    
    for i in range(n_windows):
        X[i] = decoded_sample[window_len * i: window_len * i + window_len, 0: 3]
        
        ones = np.count_nonzero(decoded_sample[window_len*i: window_len*i+window_len, 3])
        if ones >= (window_len / 2):
            y[i] = 1
        else:
            y[i] = 0
                
    return X, y

#-------------------------------------------------------------------------

def get_one_patient_data(data_path, patient, window_len=60):
    
    """
    Returns:
    X, y - for one patient
    """
    
    sample = np.load("%s/p%s.npy"%(data_path, patient)).view(np.recarray)
    sample = change_labels(sample)
    sample = decoder(sample)
    X, y = divide_by_windows(sample, window_len)
    
    return X, y

#-------------------------------------------------------------------------

def get_data_for_model(data_path, patient_list, window_len=60):
    
    """
    Returns:
    X, y - for all patient list, ndarray of shape (n_records, n_features, n_channels=3)
    """
    
    X_all_data = []
    y_all_data = []
    for patient in patient_list:
        X, y = get_one_patient_data(data_path, patient, window_len)
        X_all_data.append(X)
        y_all_data.append(y)
        
    X_all_data = np.concatenate(X_all_data, axis=0)
    y_all_data = np.concatenate(y_all_data, axis=0)
    
    return X_all_data, y_all_data
  
#-------------------------------------------------------------------------

def get_dawnsampled_data(data_path, patient_list, window_len=60, dawnsample="pca", n_components=10, n_windows=10):
    
    """
    Parameters:
    dawnsample - "pca", "mean", "max", "mode", None - determine the type of data reducing
    Returns:
    X, y - reduced data for all patient list and combine several windows data, ndarray of shape (n_records, n_components * n_windows, n_channels=3)
    """
    
    X_all_data = []
    y_all_data = []
    for patient in patient_list:
        X, y = get_one_patient_data(data_path, patient, window_len)
        
        if dawnsample.lower() == "pca":
          X = reduce_data_pca(X, n_components=n_components)
          
        elif dawnsample.lower() == "mean":
          X = reduce_data_mean(X, n_components=n_components)
          
        elif dawnsample.lower() == "max":
          X = reduce_data_max(X, n_components=n_components)
          
        elif dawnsample.lower() == "mode":
          X = reduce_data_mode(X, n_components=n_components)
          
        elif dawnsample.lower() == "simple":
          X = reduce_data_simple(X, n_components=n_components)
        
        
        X_new = np.zeros((X.shape[0] - n_windows, X.shape[1] * (n_windows + 1), X.shape[2]))
        
        for i in range(0, X.shape[0] - n_windows):
            X_buff = X[i]
            for j in range(1, n_windows + 1):
                X_buff = np.concatenate([X_buff, X[i+j]], axis=0)
            X_new[i] = X_buff                            
    
    
        if n_windows != 0:
          y = y[(n_windows//2): -(n_windows//2)]
      
        
        X_all_data.append(X_new)
        y_all_data.append(y)

        #np.save(("X_p%s.npy"%(patient)), X_new)
        #np.save(("y_p%s.npy"%(patient)), y)
        
    X_all_data = np.concatenate(X_all_data, axis=0)
    y_all_data = np.concatenate(y_all_data, axis=0)
    
    
    
    return X_all_data, y_all_data
  
def reduce_data_pca(X, n_components=300):
    """
    Parameters:
    X - ndarray of shape (n_samples, n_features)
    
    Returns:
    X, y - reduced data, ndarray of shape (n_records, n_features, n_channels=3)
    """
    pca1 = PCA(n_components)
    pca2 = PCA(n_components)
    pca3 = PCA(n_components)
    
    pca1.fit(X[:, :, 0])
    pca2.fit(X[:, :, 1])
    pca3.fit(X[:, :, 2])
    
    X1 = pca1.transform(X[:, :, 0])
    X2 = pca2.transform(X[:, :, 1])
    X3 = pca3.transform(X[:, :, 2])
    
    X_reduced = np.concatenate([X1, X2, X3], axis=1).reshape(X.shape[0], n_components, 3)
    
    return X_reduced
  
  
def reduce_data_max(X, n_components=600):
    """
    Parameters:
    X - ndarray of shape (n_samples, n_features)
    
    Returns:
    X, y - reduced data, ndarray of shape (n_records, n_components, n_channels=3)
    """
   
    
    X_reduced = np.zeros((X.shape[0], n_components, 3))
    window_len = X.shape[1] // n_components
    
    
    for i in range(n_components):
      
      X_reduced[:, i, :] = np.amax(X[:, i * window_len: (i + 1) * window_len, :], axis=1)
      
    
    X_reduced = X_reduced.reshape(X.shape[0], n_components, 3)
    
    return X_reduced
  

def reduce_data_mean(X, n_components=600):
    """
    Parameters:
    X - ndarray of shape (n_samples, n_features)
    
    Returns:
    X, y - reduced data, ndarray of shape (n_records, n_components, n_channels=3)
    """
   
    
    X_reduced = np.zeros((X.shape[0], n_components, 3))
    window_len = X.shape[1] // n_components
    
    
    for i in range(n_components):
      
      X_reduced[:, i, :] = np.mean(X[:, i * window_len: (i + 1) * window_len, :], axis=1)
         
    X_reduced = X_reduced.reshape(X.shape[0], n_components, 3)
    
    return X_reduced
  
    
def reduce_data_mode(X, n_components=600):
    """
    Parameters:
    X - ndarray of shape (n_samples, n_features)
    
    Returns:
    X, y - reduced data, ndarray of shape (n_records, n_components, n_channels=3)
    """
    
    from scipy.stats import mode
   
    X_reduced = np.zeros((X.shape[0], n_components, 3))
    window_len = X.shape[1] // n_components
       
    for i in range(n_components):
      
      X_reduced[:, i, :] = mode(X[:, i * window_len: (i + 1) * window_len, :], axis=1)
         
    X_reduced = X_reduced.reshape(X.shape[0], n_components, 3)
    
    return X_reduced
  
def reduce_data_simple(X, n_components=600):
    """
    Parameters:
    X - ndarray of shape (n_samples, n_features)
    
    Returns:
    X, y - reduced data, ndarray of shape (n_records, n_components, n_channels=3)
    """
   
    X_reduced = np.zeros((X.shape[0], n_components, 3))
    window_len = X.shape[1] // n_components
       
    for i in range(n_components):
      
      X_reduced[:, i, :] = X[:, i * window_len, :]
         
    X_reduced = X_reduced.reshape(X.shape[0], n_components, 3)
    
    return X_reduced

In [0]:
X_train, y_train = get_data_for_model(data_path, train_patient_list, window_len=60)
X_valid, y_valid = get_data_for_model(data_path, valid_patient_list, window_len=60)
X_test, y_test = get_data_for_model(data_path, test_patient_list, window_len=60)

In [9]:
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

(15835, 6000, 3)
(3565, 6000, 3)
(3805, 6000, 3)


In [10]:
%%time
X_train, y_train = get_dawnsampled_data(data_path, train_patient_list, window_len=60, dawnsample="pca", n_components=60, n_windows=12)
X_valid, y_valid = get_dawnsampled_data(data_path, valid_patient_list, window_len=60, dawnsample="pca", n_components=60, n_windows=12)
X_test, y_test = get_dawnsampled_data(data_path, test_patient_list, window_len=60, dawnsample="pca", n_components=60, n_windows=12)

CPU times: user 2min 10s, sys: 1min 13s, total: 3min 23s
Wall time: 1min 47s


In [11]:
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(X_test.shape)

(15463, 780, 3)
(15463,)
(3481, 780, 3)
(3721, 780, 3)


In [12]:
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2

from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [17]:
NN = Sequential()

NN.add(Conv1D( 32, 10, input_shape=(780, 3), activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.1)))
NN.add(BatchNormalization())
NN.add(Dropout(0.5))
NN.add(Conv1D( 32, 10, activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.1)))
NN.add(BatchNormalization())
NN.add(MaxPooling1D( pool_size=4))
NN.add(Dropout(0.5))

NN.add(Conv1D( 64, 10, activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.1)))
NN.add(BatchNormalization())
NN.add(Dropout(0.5))
NN.add(Conv1D( 64, 10, activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.1)))
NN.add(BatchNormalization())
NN.add(MaxPooling1D( pool_size=4))
NN.add(Dropout(0.5))

NN.add(Conv1D( 128, 10, activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.1)))
NN.add(BatchNormalization())
NN.add(Dropout(0.5))
NN.add(Conv1D( 128, 10, activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.1)))
NN.add(BatchNormalization())
NN.add(MaxPooling1D( pool_size=4))
NN.add(Dropout(0.5))
NN.add(Flatten())

NN.add(Dense(16, activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.1)))
NN.add(BatchNormalization())
NN.add(Dropout(0.5))

NN.add(Dense(16, activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.1)))
NN.add(BatchNormalization())
NN.add(Dropout(0.5))
NN.add(Dense(1, activation="sigmoid", kernel_initializer="glorot_uniform", kernel_regularizer=l2(0.1)))

NN.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

print(NN.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 771, 32)           992       
_________________________________________________________________
batch_normalization_9 (Batch (None, 771, 32)           128       
_________________________________________________________________
dropout_9 (Dropout)          (None, 771, 32)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 762, 32)           10272     
_________________________________________________________________
batch_normalization_10 (Batc (None, 762, 32)           128       
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 190, 32)           0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 190, 32)           0         
__________

In [0]:
callbacks = [ModelCheckpoint('CNN_model_raw_data_weights.hdf5', monitor='val_acc', save_best_only=True), EarlyStopping(monitor='val_loss', patience=5)]

In [26]:
%%time

NN.fit(X_train, y_train,
       batch_size=64, 
       epochs=30, 
       validation_data=(X_valid, y_valid), 
       callbacks=callbacks,
       verbose=1)

Train on 15463 samples, validate on 3481 samples
Epoch 1/30
15463/15463 [==============================] - 19s 1ms/step - loss: 0.7917 - acc: 0.7259 - val_loss: 0.7647 - val_acc: 0.7449
Epoch 2/30
15463/15463 [==============================] - 19s 1ms/step - loss: 0.7950 - acc: 0.7239 - val_loss: 0.8417 - val_acc: 0.6880
Epoch 3/30
15463/15463 [==============================] - 19s 1ms/step - loss: 0.8385 - acc: 0.7222 - val_loss: 0.9217 - val_acc: 0.6475
Epoch 4/30
15463/15463 [==============================] - 19s 1ms/step - loss: 0.8180 - acc: 0.7191 - val_loss: 0.7843 - val_acc: 0.6808
Epoch 5/30
15463/15463 [==============================] - 18s 1ms/step - loss: 0.8081 - acc: 0.7222 - val_loss: 0.8054 - val_acc: 0.7193
Epoch 6/30
15463/15463 [==============================] - 19s 1ms/step - loss: 0.8150 - acc: 0.7252 - val_loss: 0.8574 - val_acc: 0.7366
CPU times: user 1min 24s, sys: 23.4 s, total: 1min 47s
Wall time: 1min 52s


In [20]:
scores = NN.evaluate(X_test, y_test)
print("Test accuracy =", scores[1])

3721/3721 [==============================] - 2s 632us/step
Test accuracy = 0.7245364149422199


In [0]:
NN.save_weights("CNN_12w_pca60_raw_data_weights.hdf5")

In [0]:
files.download('CNN_model_raw_data_weights.hdf5')

In [0]:
# Load best model
NN.load_weights("CNN_model_raw_data_weights.hdf5")

In [23]:
scores = NN.evaluate(X_test, y_test)
print("Test accuracy =", scores[1])

3721/3721 [==============================] - 1s 342us/step
Test accuracy = 0.7245364149422199


In [24]:
scores = NN.evaluate(X_valid, y_valid)
print("Valid accuracy =", scores[1])

3481/3481 [==============================] - 1s 341us/step
Valid accuracy = 0.7302499281901185


In [0]:
saved_model = NN.to_json()
with open("CNN_model_raw_data.json", "w") as json_file:
    json_file.write(saved_model)
    
files.download('CNN_model_raw_data.json')

pca 300, 10 windows: max test acc = 70% , ~20 min, ~ 30 epoch, EarlyStopping = 5

max 300, 10 windows: max test acc = 0.6996, ~30 min, ~40 epoch, EarlyStopping = 10

mean 300, 10 windows: max test acc = 0.5449, ~20 min, ~28 epoch, EarlyStopping = 5

pca 350, 10 windows: max test acc = 0.698, ~10min, ~18 epoch, EarlyStopping = 5

pca 60, 10 windows: max test acc = 0.7256, 2 mi, 11, epoch, EarlyStopping = 5

pca 60, 20 windows: max test acc =0.71, 5 min, 17 epoch, EarlyStopping = 5 , file 4

pca 60, 60 windows: max test acc =0.7243, 7 min, 10 epoch, EarlyStopping = 5 , file 6

pca 120, 16 windows: max test acc =0.7070, 9 min, 21 epoch, EarlyStopping = 7 , file 8

after strong regularization

pca 60, 12 windows: max test acc =0.7269, 10 min, 30 epoch, EarlyStopping = 10

pca 300, 20 windows: max test acc = 0.6881, ~20 min, ~15 epoch, EarlyStopping = 10

pca 60, 20 windows: max test acc =0.7222, 9 min, 25 epoch, EarlyStopping = 10
